In [1]:
import pandas as pd
import numpy as np
from mi import mutual_info
import entropy_estimators as EE
from data_loader import load_data

In [2]:
import scipy
def entropy(x, k=3, base=2):
    assert k < x.shape[0] - 1
    d = x.shape[1]
    N = x.shape[0]
    intens = 1e-10
    x_ = x + intens*np.random.rand(N, d)
    tree = scipy.spatial.cKDTree(x_)
    nn = tree.query(x_, k+1, p=float('inf'))[0][:, k]
    const = scipy.special.digamma(N) - scipy.special.digamma(k) + d*np.log(2)
    return (const + d*np.mean(np.log(nn))) / np.log(base)
    
def micd(x, y, k=3, base=2, warning=True):
    # I(x, y) = H(x) + sum_y p(Y == y) H(x | Y == y)
    assert x.shape[0] == y.shape[0]
    assert len(y.shape) == 1
    overallentropy = entropy(x, k, base)
    
    classes = np.unique(y)
    
    mi = overallentropy
    for c in range(classes.shape[0]):
        x_given_y = x[y == c]
        num_c = x_given_y.shape[0]
        p_c = num_c/x.shape[0]
        if k < num_c:
            mi -= p_c * entropy(x_given_y, k, base)
        else:
            if warning:
                print("Warning, after condition on y=", c, ", insufficient data. Assuming maximal entropy.")
            mi -= p_c * overallentropy
    return np.abs(mi)



In [3]:
valid = load_data("/home/dillon/data/results/elmoSEQ/elmoSEQ_sub1_valid.pkl")
test = load_data("/home/dillon/data/results/elmoSEQ/elmoSEQ_sub1_test.pkl")
data = pd.concat([valid, test])
print(data.shape)

(200173, 13)


In [4]:
pssm = np.concatenate([data.pssm.iloc[i].reshape(1, -1) for i in range(data.shape[0])])
print(pssm.shape)
ss = np.array(data.ss.apply(np.argmax).values)
print(ss.shape)
hs = [np.concatenate([data[h_i].iloc[i].reshape(1, -1) for i in range(data.shape[0])]) for h_i in ["h_0", "h_1", "h_2", "lm_logits"]]
print(hs[0].shape)

(200173, 21)
(200173,)
(200173, 512)


In [5]:
mi_pssm_ss = micd(pssm, ss)
mi_pssm_ss

6.495771245700312

In [6]:
mi_logits_ss = micd(hs[3], ss)
mi_logits_ss

1.2591490151685534

In [ ]:
mi_h0_ss = micd(hs[0], ss)
mi_h0_ss

In [ ]:
mi_h1_ss = micd(hs[1], ss)
mi_h1_ss

In [ ]:
mi_h2_ss = micd(hs[2], ss)
mi_h2_ss

In [ ]:
x = np.random.rand(100, 20)
list_x = [list(x[i]) for i in range(x.shape[0])]
%timeit EE.entropy(list_x)
%timeit entropy(x)

In [ ]:
y = np.random.randint(0, 5, size=100)
list_y = [[y[i]] for i in range(y.shape[0])]
%timeit EE.micd(list_x, list_y)
%timeit micd(x, y)

In [ ]:
h = np.concatenate(hs+[pssm], axis=1)
print(h.shape)

In [ ]:
cmi_h_ss_pssm = micd(h, ss) - mi_pssm_ss
cmi_h_ss_pssm